### Section 1: Data Loading and Preprocessing 

In [1]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [5]:
# defining and downloading the AAPL ticker for analysis.
ticker = "AAPL"

# Download 10 years of daily data
data = yf.download(
    ticker,
    period="10y",     # last 10 years
    interval="1d",    # daily data
    auto_adjust=False # keep raw OHLC, you can set True if you want adjusted prices
)

# Quick check
print(data.head())
print(data.tail())
print(data.shape)

[*********************100%***********************]  1 of 1 completed

Price       Adj Close      Close       High      Low       Open     Volume
Ticker           AAPL       AAPL       AAPL     AAPL       AAPL       AAPL
Date                                                                      
2015-11-27  26.562494  29.452499  29.602501  29.4000  29.572500   52185600
2015-11-30  26.672977  29.575001  29.852501  29.4375  29.497499  156721200
2015-12-01  26.456518  29.334999  29.702499  29.2150  29.687500  139409600
2015-12-02  26.217531  29.070000  29.527500  29.0200  29.334999  133546400
2015-12-03  25.974020  28.799999  29.197500  28.5550  29.137501  166278000
Price        Adj Close       Close        High         Low        Open  \
Ticker            AAPL        AAPL        AAPL        AAPL        AAPL   
Date                                                                     
2025-11-20  266.250000  266.250000  275.429993  265.920013  270.829987   
2025-11-21  271.489990  271.489990  273.329987  265.670013  265.950012   
2025-11-24  275.920013  275.92

In [ ]:
print(data.columns)
print()
print(data.index)

MultiIndex([('Adj Close', 'AAPL'),
            (    'Close', 'AAPL'),
            (     'High', 'AAPL'),
            (      'Low', 'AAPL'),
            (     'Open', 'AAPL'),
            (   'Volume', 'AAPL')],
           names=['Price', 'Ticker'])

DatetimeIndex(['2015-11-27', '2015-11-30', '2015-12-01', '2015-12-02',
               '2015-12-03', '2015-12-04', '2015-12-07', '2015-12-08',
               '2015-12-09', '2015-12-10',
               ...
               '2025-11-13', '2025-11-14', '2025-11-17', '2025-11-18',
               '2025-11-19', '2025-11-20', '2025-11-21', '2025-11-24',
               '2025-11-25', '2025-11-26'],
              dtype='datetime64[ns]', name='Date', length=2515, freq=None)


In [ ]:
aapl = data.copy()

# keep just the first level (Open, High, Low, Close, Volume)
aapl.columns = aapl.columns.droplevel(1)

# Remove the "Price" label on the columns index
aapl.columns.name = None

# Turn the Date index into a normal column
aapl = aapl.reset_index()      # Date becomes a column

# Keep only the columns you care about
aapl = aapl[["Date", "Open", "High", "Low", "Close", "Volume", "Adj Close"]]

print(aapl.head())
print(aapl.dtypes)
print(aapl.shape)


        Date       Open       High      Low      Close     Volume  Adj Close
0 2015-11-27  29.572500  29.602501  29.4000  29.452499   52185600  26.562494
1 2015-11-30  29.497499  29.852501  29.4375  29.575001  156721200  26.672977
2 2015-12-01  29.687500  29.702499  29.2150  29.334999  139409600  26.456518
3 2015-12-02  29.334999  29.527500  29.0200  29.070000  133546400  26.217531
4 2015-12-03  29.137501  29.197500  28.5550  28.799999  166278000  25.974020
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Volume                int64
Adj Close           float64
dtype: object
(2515, 7)


In [30]:
aapl["daily_ret"] = aapl["Adj Close"].pct_change()

aapl = aapl.dropna(subset=['daily_ret']).reset_index(drop=True)

print(aapl.head())
# print(aapl.isna().sum())

        Date       Open       High      Low      Close     Volume  Adj Close  \
0 2015-11-30  29.497499  29.852501  29.4375  29.575001  156721200  26.672977   
1 2015-12-01  29.687500  29.702499  29.2150  29.334999  139409600  26.456518   
2 2015-12-02  29.334999  29.527500  29.0200  29.070000  133546400  26.217531   
3 2015-12-03  29.137501  29.197500  28.5550  28.799999  166278000  25.974020   
4 2015-12-04  28.822500  29.812500  28.7775  29.757500  231108000  26.837566   

   daily_ret  
0   0.004159  
1  -0.008115  
2  -0.009033  
3  -0.009288  
4   0.033247  


In [32]:
aapl.to_csv("AAPL_clean_2015-11-27_2025-11-27.csv", index=False)

In [34]:
aapl.shape

(2514, 8)

In [2]:
# Loading the cleaned data
aapl_clean = pd.read_csv("AAPL_clean_2015-11-27_2025-11-27.csv")
print(aapl_clean.head())

         Date       Open       High      Low      Close     Volume  Adj Close  \
0  2015-11-30  29.497499  29.852501  29.4375  29.575001  156721200  26.672977   
1  2015-12-01  29.687500  29.702499  29.2150  29.334999  139409600  26.456518   
2  2015-12-02  29.334999  29.527500  29.0200  29.070000  133546400  26.217531   
3  2015-12-03  29.137501  29.197500  28.5550  28.799999  166278000  25.974020   
4  2015-12-04  28.822500  29.812500  28.7775  29.757500  231108000  26.837566   

   daily_ret  
0   0.004159  
1  -0.008115  
2  -0.009033  
3  -0.009288  
4   0.033247  


In [3]:
#  Load TD file and use 'Return' column
fund = pd.read_csv("TDB908.csv")

fund["Date"] = pd.to_datetime(fund["Date"])
fund = fund.sort_values("Date")

# 'Return' is like "0.87%" -> convert to decimal (0.0087)
fund["fund_ret"] = (
    fund["Return"]
    .astype(str)
    .str.replace("%", "", regex=False)
    .astype(float)
    / 100.0
)

# Equity curve starting at 1
fund["fund_equity"] = (1 + fund["fund_ret"].fillna(0)).cumprod()

fund.head()

,Date,Open,High,Low,Close,VWAP,Volume,Return,$ Chg,Trade Val,# Trades,Unnamed: 11,fund_ret,fund_equity
2510,2015-11-27,--,--,--,11.52,--,--,0.16%,0.018778,--,--,NaN,0.0016,1.001600
2509,2015-11-30,--,--,--,11.47,--,--,-0.41%,-0.046947,--,--,NaN,-0.0041,0.997493
2508,2015-12-01,--,--,--,11.61,--,--,1.15%,0.13145,--,--,NaN,0.0115,1.008965
2507,2015-12-02,--,--,--,11.53,--,--,-0.65%,-0.075114,--,--,NaN,-0.0065,1.002406
2506,2015-12-03,--,--,--,11.33,--,--,-1.71%,-0.197173,--,--,NaN,-0.0171,0.985265


# Section 2: Defining functions to calculate performance metrics 


In [4]:
import numpy as np

TRADING_DAYS = 252         # trading days per year
RISK_FREE_ANNUAL = 0.02    # 2% annual risk-free (change if you want)


def success_rate(returns: pd.Series) -> float:
    """
    Fraction of periods with positive return.
    returns: series of periodic returns (e.g., daily)
    """
    returns = returns.dropna()
    if len(returns) == 0:
        return 0.0
    wins = (returns > 0).sum()
    return wins / len(returns)


def total_and_annual_return(equity: pd.Series,
                            periods_per_year: int = TRADING_DAYS):
    """
    Computes total return and annualized return from an equity curve.
    equity: portfolio value over time (e.g., starts at 1.0)
    """
    equity = equity.dropna()
    if len(equity) < 2:
        return 0.0, 0.0

    total_ret = equity.iloc[-1] / equity.iloc[0] - 1.0

    n_periods = len(equity) - 1
    if n_periods <= 0:
        return total_ret, 0.0

    ann_ret = (1.0 + total_ret) ** (periods_per_year / n_periods) - 1.0
    return total_ret, ann_ret


def sharpe_ratio(returns: pd.Series,
                 risk_free_annual: float = RISK_FREE_ANNUAL,
                 periods_per_year: int = TRADING_DAYS) -> float:
    """
    Annualized Sharpe ratio using excess returns over risk-free rate.
    returns: periodic returns (e.g., daily)
    """
    returns = returns.dropna()
    if len(returns) == 0:
        return np.nan

    # Convert annual risk-free rate to per-period
    rf_period = (1.0 + risk_free_annual) ** (1.0 / periods_per_year) - 1.0
    excess = returns - rf_period

    mean_excess = excess.mean()
    std_excess = excess.std(ddof=1)

    if std_excess == 0:
        return np.nan

    sharpe = (mean_excess / std_excess) * np.sqrt(periods_per_year)
    return sharpe


def summarize_strategy(name, ret_series, equity_series):
    """Helper to compute all metrics for one strategy."""
    sr   = success_rate(ret_series)
    tot, ann = total_and_annual_return(equity_series)
    sh   = sharpe_ratio(ret_series)

    return {
        "Strategy": name,
        "Success Rate (%)": sr * 100,
        "Total Return (%)": tot * 100,
        "Annual Return (%)": ann * 100,
        "Sharpe Ratio": sh,
    }


### Analyzing metrics on Mutual Fund performance 

In [5]:
fund_sr              = success_rate(fund["fund_ret"])
fund_tot, fund_ann   = total_and_annual_return(fund["fund_equity"])
fund_sharpe          = sharpe_ratio(fund["fund_ret"])

print("\n=== TD Nasdaq Index Fund (TDB908) ===")
print(f"Success rate (daily): {fund_sr:.2%}")
print(f"Total return        : {fund_tot:.2%}")
print(f"Annualized return   : {fund_ann:.2%}")
print(f"Sharpe ratio        : {fund_sharpe:.2f}")



=== TD Nasdaq Index Fund (TDB908) ===
Success rate (daily): 54.28%
Total return        : 393.92%
Annualized return   : 17.39%
Sharpe ratio        : 0.74


# Section 3: Technical Analysis Strategies 

### Strategy 1: Donchian breakout strategy

In [6]:
def add_donchian_atr_indicators(df,
                                upper_len=20,
                                lower_len_entry=20,
                                lower_len_exit=10,
                                atr_len=14):
    """
    Adds:
      - DC_high_20: rolling 20-day highest high (entry breakout level)
      - DC_low_20:  rolling 20-day lowest low  (context only, optional)
      - DC_low_10:  rolling 10-day lowest low  (exit)
      - ATR:        14-day Average True Range

    Expects columns: High, Low, Close.
    """
    df = df.copy()

    df["DC_high_20"] = df["High"].rolling(window=upper_len, min_periods=upper_len).max()
    df["DC_low_20"]  = df["Low"].rolling(window=lower_len_entry, min_periods=lower_len_entry).min()
    df["DC_low_10"]  = df["Low"].rolling(window=lower_len_exit, min_periods=lower_len_exit).min()

    df["ATR"] = ta.atr(high=df["High"], low=df["Low"], close=df["Close"], length=atr_len)

    return df


In [7]:
def run_donchian_breakout_strategy(df,
                                   upper_len=20,
                                   lower_len_entry=20,
                                   lower_len_exit=10,
                                   atr_len=14,
                                   atr_mult=2.0,
                                   max_holding=60,
                                   initial_capital=1.0):
    """
    Strategy 3: Donchian Channel Volatility Breakout (long-only).

    ENTRY (long):
        - Close_t > DC_high_20_(t-1)  (new 20-day breakout)

    EXIT:
        - Close_t < DC_low_10_t       (short-term breakdown)
        OR
        - Close_t < entry_price - atr_mult * ATR_at_entry  (vol stop)
        OR
        - holding_days >= max_holding  (time stop)

    Expects df to have:
      - Date, Open, High, Low, Close, Adj Close, daily_ret

    Returns df with:
      - DC_high_20, DC_low_20, DC_low_10, ATR
      - position (0/1)
      - strategy_ret
      - strategy_equity
      - buyhold_equity
    """
    df = df.copy()
    df = add_donchian_atr_indicators(df,
                                     upper_len=upper_len,
                                     lower_len_entry=lower_len_entry,
                                     lower_len_exit=lower_len_exit,
                                     atr_len=atr_len)

    # Ensure date ordering
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date").reset_index(drop=True)

    df["position"] = 0
    in_position = False
    holding_days = 0
    entry_price = np.nan
    entry_atr = np.nan

    pos_col = df.columns.get_loc("position")

    for i in range(len(df)):
        row = df.iloc[i]

        # Need valid indicators and previous DC_high_20 for breakout check
        if i == 0 or pd.isna(row["DC_high_20"]) or pd.isna(row["DC_low_10"]) or pd.isna(row["ATR"]):
            df.iloc[i, pos_col] = 1 if in_position else 0
            continue

        prev = df.iloc[i - 1]

        #  ENTRY: Donchian breakout above 20-day high
        breakout = row["Close"] > prev["DC_high_20"]

        if (not in_position) and breakout:
            in_position = True
            holding_days = 0
            entry_price = row["Close"]
            entry_atr = row["ATR"]
            df.iloc[i, pos_col] = 1

        #  EXIT: breakdown, vol stop, or time stop
        elif in_position:
            holding_days += 1

            stop_level = entry_price - atr_mult * entry_atr
            breakdown  = row["Close"] < row["DC_low_10"]
            vol_stop   = row["Close"] < stop_level
            time_exit  = holding_days >= max_holding

            if breakdown or vol_stop or time_exit:
                in_position = False
                holding_days = 0
                entry_price = np.nan
                entry_atr = np.nan
                df.iloc[i, pos_col] = 0
            else:
                df.iloc[i, pos_col] = 1

        else:
            df.iloc[i, pos_col] = 0

    # Strategy returns: yesterday's position * today's daily return
    df["strategy_ret"] = df["position"].shift(1) * df["daily_ret"]
    df["strategy_ret"] = df["strategy_ret"].fillna(0)

    # Equity curves
    df["strategy_equity"] = initial_capital * (1 + df["strategy_ret"]).cumprod()
    df["buyhold_equity"]  = initial_capital * (1 + df["daily_ret"]).cumprod()

    return df


In [8]:
donch_results = run_donchian_breakout_strategy(aapl_clean)

print(donch_results[["Date", "Close", "DC_high_20", "DC_low_10",
                     "ATR", "position",
                     "strategy_equity", "buyhold_equity"]].head())


        Date      Close  DC_high_20  DC_low_10  ATR  position  \
0 2015-11-30  29.575001         NaN        NaN  NaN         0   
1 2015-12-01  29.334999         NaN        NaN  NaN         0   
2 2015-12-02  29.070000         NaN        NaN  NaN         0   
3 2015-12-03  28.799999         NaN        NaN  NaN         0   
4 2015-12-04  29.757500         NaN        NaN  NaN         0   

   strategy_equity  buyhold_equity  
0              1.0        1.004159  
1              1.0        0.996010  
2              1.0        0.987013  
3              1.0        0.977846  
4              1.0        1.010356  


In [9]:
donch_results["Date"] = pd.to_datetime(donch_results["Date"])
donch_results = donch_results.sort_values("Date").reset_index(drop=True)

# Zoom window (years)
window_years = 2
end_date = donch_results["Date"].max()
start_date = end_date - pd.Timedelta(days=365 * window_years)

view = donch_results[
    (donch_results["Date"] >= start_date) &
    (donch_results["Date"] <= end_date)
].copy()

# Entry/exit flags
entries = (view["position"].shift(1) == 0) & (view["position"] == 1)
exits   = (view["position"].shift(1) == 1) & (view["position"] == 0)

# Subplots: top = price + channels + trades, bottom = ATR
fig_dc = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    row_heights=[0.7, 0.3],
    vertical_spacing=0.08,
    subplot_titles=(
        "AAPL Close with Donchian Channels & Breakout Trades",
        "ATR(14) – Volatility (used for stop-loss)"
    ),
)

# ----- Row 1: Price + Donchian channels -----
fig_dc.add_trace(
    go.Scatter(
        x=view["Date"],
        y=view["Close"],
        mode="lines",
        name="Close"
    ),
    row=1, col=1
)

fig_dc.add_trace(
    go.Scatter(
        x=view["Date"],
        y=view["DC_high_20"],
        mode="lines",
        name="20-day High (Entry Level)"
    ),
    row=1, col=1
)

fig_dc.add_trace(
    go.Scatter(
        x=view["Date"],
        y=view["DC_low_10"],
        mode="lines",
        name="10-day Low (Exit Level)"
    ),
    row=1, col=1
)

# Entry markers – bright green up triangles (breakouts)
fig_dc.add_trace(
    go.Scatter(
        x=view.loc[entries, "Date"],
        y=view.loc[entries, "Close"],
        mode="markers",
        name="Entry (Breakout)",
        marker=dict(
            symbol="triangle-up",
            size=14,
            color="lime",
            line=dict(width=1, color="black"),
        ),
    ),
    row=1, col=1,
)

# Exit markers – bright red down triangles
fig_dc.add_trace(
    go.Scatter(
        x=view.loc[exits, "Date"],
        y=view.loc[exits, "Close"],
        mode="markers",
        name="Exit",
        marker=dict(
            symbol="triangle-down",
            size=14,
            color="red",
            line=dict(width=1, color="black"),
        ),
    ),
    row=1, col=1,
)

# ----- Row 2: ATR -----
fig_dc.add_trace(
    go.Scatter(
        x=view["Date"],
        y=view["ATR"],
        mode="lines",
        name="ATR(14)"
    ),
    row=2, col=1
)

# Layout
fig_dc.update_layout(
    title=f"AAPL – Donchian Breakout Strategy (Zoomed: {start_date.date()} to {end_date.date()})",
    yaxis_title="Price",
    xaxis2_title="Date",
    yaxis2_title="ATR",
    legend_title="Legend",
    xaxis_rangeslider_visible=False,
    height=700,
)

fig_dc.show()


In [10]:
# fund is sorted with Date + fund_equity
fund["Date"] = pd.to_datetime(fund["Date"])
fund = fund.sort_values("Date")

merged_donch = pd.merge(
    donch_results[["Date", "strategy_equity", "buyhold_equity"]],
    fund[["Date", "fund_equity"]],
    on="Date",
    how="inner"
)

fig_eq_donch = go.Figure()

fig_eq_donch.add_trace(go.Scatter(
    x=merged_donch["Date"],
    y=merged_donch["strategy_equity"],
    mode="lines",
    name="Donchian Breakout Strategy"
))

fig_eq_donch.add_trace(go.Scatter(
    x=merged_donch["Date"],
    y=merged_donch["buyhold_equity"],
    mode="lines",
    name="Buy & Hold AAPL"
))

fig_eq_donch.add_trace(go.Scatter(
    x=merged_donch["Date"],
    y=merged_donch["fund_equity"],
    mode="lines",
    name="TDB908 Fund"
))

fig_eq_donch.update_layout(
    title="Equity Curves – Donchian Breakout vs AAPL vs TD Nasdaq Index Fund",
    xaxis_title="Date",
    yaxis_title="Portfolio value (start = 1.0)",
    legend_title="Legend",
    xaxis_rangeslider_visible=False,
    height=600,
)

fig_eq_donch.show()


### Strategy: Supertrend Trend-Following (ATR-based)

In [11]:
def add_supertrend(df, length=10, multiplier=3.0):
    """
    Adds Supertrend (trend-following volatility band) to a copy of df.
    Uses High, Low, Close.

    Creates:
      - 'SUPERT': Supertrend value (line)
      - 'SUPERT_DIR': Trend direction (+1 uptrend, -1 downtrend)
    """
    df = df.copy()

    st = ta.supertrend(
        high=df["High"],
        low=df["Low"],
        close=df["Close"],
        length=length,
        multiplier=multiplier
    )
    # pandas_ta usually returns columns like: SUPERT_10_3.0, SUPERTd_10_3.0
    df["SUPERT"] = st.iloc[:, 0]     # Supertrend line
    df["SUPERT_DIR"] = st.iloc[:, 1] # Trend direction

    return df


In [12]:
def run_supertrend_strategy(df,
                            length=10,
                            multiplier=3.0,
                            initial_capital=1.0):
    """
    Strategy A: Supertrend Trend-Following (long-only)

    ENTRY:
        - Close crosses ABOVE Supertrend
          (price above volatility-adjusted trailing line)

    EXIT:
        - Close crosses BELOW Supertrend
          (trend breaks)

    Expects df to have:
      - Date, Open, High, Low, Close, Adj Close, daily_ret

    Returns df with:
      - SUPERT, SUPERT_DIR
      - position (0/1)
      - strategy_ret
      - strategy_equity
      - buyhold_equity
    """
    df = df.copy()
    df = add_supertrend(df, length=length, multiplier=multiplier)

    # Ensure proper ordering
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date").reset_index(drop=True)

    df["position"] = 0
    in_position = False
    pos_col = df.columns.get_loc("position")

    for i in range(len(df)):
        row = df.iloc[i]

        # Need at least one previous bar and a valid Supertrend value
        if i == 0 or pd.isna(row["SUPERT"]):
            df.iloc[i, pos_col] = 1 if in_position else 0
            continue

        prev = df.iloc[i - 1]

        # Crosses above Supertrend → entry
        cross_up = (prev["Close"] <= prev["SUPERT"]) and (row["Close"] > row["SUPERT"])

        # Crosses below Supertrend → exit
        cross_down = (prev["Close"] >= prev["SUPERT"]) and (row["Close"] < row["SUPERT"])

        # ----- ENTRY -----
        if (not in_position) and cross_up:
            in_position = True
            df.iloc[i, pos_col] = 1

        # ----- EXIT -----
        elif in_position and cross_down:
            in_position = False
            df.iloc[i, pos_col] = 0

        # ----- HOLD / FLAT -----
        else:
            df.iloc[i, pos_col] = 1 if in_position else 0

    # Strategy daily returns = yesterday's position * today's daily return
    df["strategy_ret"] = df["position"].shift(1) * df["daily_ret"]
    df["strategy_ret"] = df["strategy_ret"].fillna(0)

    # Equity curves
    df["strategy_equity"] = initial_capital * (1 + df["strategy_ret"]).cumprod()
    df["buyhold_equity"]  = initial_capital * (1 + df["daily_ret"]).cumprod()

    return df


In [13]:
super_results = run_supertrend_strategy(aapl_clean)
super_results["Date"] = pd.to_datetime(super_results["Date"])
super_results = super_results.sort_values("Date").reset_index(drop=True)

# Choose zoom window (years)
window_years = 2
end_date = super_results["Date"].max()
start_date = end_date - pd.Timedelta(days=365 * window_years)

view_super = super_results[
    (super_results["Date"] >= start_date) &
    (super_results["Date"] <= end_date)
].copy()

# Entry/exit flags (based on position changes)
entries_super = (view_super["position"].shift(1) == 0) & (view_super["position"] == 1)
exits_super   = (view_super["position"].shift(1) == 1) & (view_super["position"] == 0)

# ---------- Plot: Price + Supertrend + signals ----------
fig_super = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    row_heights=[0.7, 0.3],
    vertical_spacing=0.08,
    subplot_titles=(
        "AAPL Close with Supertrend & Trade Signals",
        "Supertrend Direction (+1 Uptrend, -1 Downtrend)"
    ),
)

# Row 1 – price + Supertrend
fig_super.add_trace(
    go.Scatter(
        x=view_super["Date"],
        y=view_super["Close"],
        mode="lines",
        name="Close"
    ),
    row=1, col=1
)

fig_super.add_trace(
    go.Scatter(
        x=view_super["Date"],
        y=view_super["SUPERT"],
        mode="lines",
        name="Supertrend"
    ),
    row=1, col=1
)

# Entry markers – bright green triangles
fig_super.add_trace(
    go.Scatter(
        x=view_super.loc[entries_super, "Date"],
        y=view_super.loc[entries_super, "Close"],
        mode="markers",
        name="Entry",
        marker=dict(
            symbol="triangle-up",
            size=14,
            color="lime",
            line=dict(width=1, color="black"),
        ),
    ),
    row=1, col=1,
)

# Exit markers – bright red triangles
fig_super.add_trace(
    go.Scatter(
        x=view_super.loc[exits_super, "Date"],
        y=view_super.loc[exits_super, "Close"],
        mode="markers",
        name="Exit",
        marker=dict(
            symbol="triangle-down",
            size=14,
            color="red",
            line=dict(width=1, color="black"),
        ),
    ),
    row=1, col=1,
)

# Row 2 – Supertrend direction (+1 / -1)
fig_super.add_trace(
    go.Scatter(
        x=view_super["Date"],
        y=view_super["SUPERT_DIR"],
        mode="lines",
        name="Supertrend Dir"
    ),
    row=2, col=1
)

fig_super.update_layout(
    title=f"AAPL – Supertrend Strategy (Zoomed: {start_date.date()} to {end_date.date()})",
    yaxis_title="Price",
    xaxis2_title="Date",
    yaxis2_title="Direction",
    legend_title="Legend",
    xaxis_rangeslider_visible=False,
    height=700,
)

fig_super.show()

In [14]:
fund["Date"] = pd.to_datetime(fund["Date"])
fund = fund.sort_values("Date")

merged_super = pd.merge(
    super_results[["Date", "strategy_equity", "buyhold_equity"]],
    fund[["Date", "fund_equity"]],
    on="Date",
    how="inner"
)

fig_eq_super = go.Figure()

fig_eq_super.add_trace(go.Scatter(
    x=merged_super["Date"],
    y=merged_super["strategy_equity"],
    mode="lines",
    name="Supertrend Strategy"
))

fig_eq_super.add_trace(go.Scatter(
    x=merged_super["Date"],
    y=merged_super["buyhold_equity"],
    mode="lines",
    name="Buy & Hold AAPL"
))

fig_eq_super.add_trace(go.Scatter(
    x=merged_super["Date"],
    y=merged_super["fund_equity"],
    mode="lines",
    name="TDB908 Fund"
))

fig_eq_super.update_layout(
    title="Equity Curves – Supertrend Strategy vs AAPL vs TD Nasdaq Index Fund",
    xaxis_title="Date",
    yaxis_title="Portfolio value (start = 1.0)",
    legend_title="Legend",
    xaxis_rangeslider_visible=False,
    height=600,
)

fig_eq_super.show()


### Strategy: Parabolic SAR Trend Strategy

In [15]:
def add_psar(df, step=0.02, max_step=0.2):
    """
    Adds a unified Parabolic SAR series to df.

    pandas_ta.psar returns (usually):
      - long SAR column (dots below price)
      - short SAR column (dots above price)

    We combine them into a single 'PSAR' column:
      PSAR = long_SAR where available, else short_SAR.
    """
    df = df.copy()

    psar = ta.psar(
        high=df["High"],
        low=df["Low"],
        close=df["Close"],
        step=step,
        max_step=max_step,
    )

    # Inspect psar.columns if you want, but typically:
    # col 0 = PSARl_..., col 1 = PSARs_..., col 2 = something like PSAR_...
    long_sar  = psar.iloc[:, 0]
    short_sar = psar.iloc[:, 1]

    # Unified PSAR: use long when available, otherwise short
    df["PSAR"] = long_sar.fillna(short_sar)

    return df



In [16]:
def run_psar_trend_strategy(df,
                            step=0.02,
                            max_step=0.2,
                            initial_capital=1.0):
    """
    Parabolic SAR Trend Strategy (long-only).

    ENTRY:
        - PSAR flips from ABOVE price to BELOW price:
          PSAR_{t-1} >= Close_{t-1} and PSAR_t < Close_t

    EXIT:
        - PSAR flips from BELOW price to ABOVE price:
          PSAR_{t-1} <= Close_{t-1} and PSAR_t > Close_t

    Expects df to have:
      - Date, Open, High, Low, Close, daily_ret

    Returns df with:
      - PSAR
      - position (0/1)
      - strategy_ret
      - strategy_equity
      - buyhold_equity
    """
    df = df.copy()
    df = add_psar(df, step=step, max_step=max_step)

    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date").reset_index(drop=True)

    df["position"] = 0
    in_position = False
    pos_col = df.columns.get_loc("position")

    for i in range(len(df)):
        row = df.iloc[i]

        # Need a previous bar and valid PSAR
        if i == 0 or pd.isna(row["PSAR"]):
            df.iloc[i, pos_col] = 1 if in_position else 0
            continue

        prev = df.iloc[i - 1]

        # Flip to uptrend: PSAR crosses from above to below price
        flip_up   = (prev["PSAR"] >= prev["Close"]) and (row["PSAR"] < row["Close"])
        # Flip to downtrend: PSAR crosses from below to above price
        flip_down = (prev["PSAR"] <= prev["Close"]) and (row["PSAR"] > row["Close"])

        # ENTRY
        if (not in_position) and flip_up:
            in_position = True
            df.iloc[i, pos_col] = 1

        # EXIT
        elif in_position and flip_down:
            in_position = False
            df.iloc[i, pos_col] = 0

        # HOLD / FLAT
        else:
            df.iloc[i, pos_col] = 1 if in_position else 0

    # Strategy daily returns
    df["strategy_ret"] = df["position"].shift(1) * df["daily_ret"]
    df["strategy_ret"] = df["strategy_ret"].fillna(0)

    # Equity curves
    df["strategy_equity"] = initial_capital * (1 + df["strategy_ret"]).cumprod()
    df["buyhold_equity"]  = initial_capital * (1 + df["daily_ret"]).cumprod()

    return df


In [17]:
psar_results = run_psar_trend_strategy(aapl_clean)

print(psar_results["position"].value_counts())
print(psar_results["strategy_ret"].describe())


position
1    1487
0    1027
Name: count, dtype: int64
count    2514.000000
mean        0.000650
std         0.011902
min        -0.099607
25%        -0.000867
50%         0.000000
75%         0.003474
max         0.087237
Name: strategy_ret, dtype: float64


In [18]:

psar_results["Date"] = pd.to_datetime(psar_results["Date"])
psar_results = psar_results.sort_values("Date").reset_index(drop=True)

window_years = 2
end_date = psar_results["Date"].max()
start_date = end_date - pd.Timedelta(days=365 * window_years)

view_psar = psar_results[
    (psar_results["Date"] >= start_date) &
    (psar_results["Date"] <= end_date)
].copy()

entries_psar = (view_psar["position"].shift(1) == 0) & (view_psar["position"] == 1)
exits_psar   = (view_psar["position"].shift(1) == 1) & (view_psar["position"] == 0)

# ---------- Plot: price + PSAR ----------
fig_psar = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    row_heights=[0.7, 0.3],
    vertical_spacing=0.08,
    subplot_titles=(
        "AAPL Close with Parabolic SAR & Trade Signals",
        "Parabolic SAR Value"
    ),
)

# Row 1 – price
fig_psar.add_trace(
    go.Scatter(
        x=view_psar["Date"],
        y=view_psar["Close"],
        mode="lines",
        name="Close"
    ),
    row=1, col=1
)

# PSAR dots on top panel (to see relation to price)
fig_psar.add_trace(
    go.Scatter(
        x=view_psar["Date"],
        y=view_psar["PSAR"],
        mode="markers",
        name="PSAR",
        marker=dict(
            size=6,
            color="orange"
        ),
    ),
    row=1, col=1
)

# Entry markers – bright green triangles
fig_psar.add_trace(
    go.Scatter(
        x=view_psar.loc[entries_psar, "Date"],
        y=view_psar.loc[entries_psar, "Close"],
        mode="markers",
        name="Entry",
        marker=dict(
            symbol="triangle-up",
            size=14,
            color="lime",
            line=dict(width=1, color="black"),
        ),
    ),
    row=1, col=1,
)

# Exit markers – bright red triangles
fig_psar.add_trace(
    go.Scatter(
        x=view_psar.loc[exits_psar, "Date"],
        y=view_psar.loc[exits_psar, "Close"],
        mode="markers",
        name="Exit",
        marker=dict(
            symbol="triangle-down",
            size=14,
            color="red",
            line=dict(width=1, color="black"),
        ),
    ),
    row=1, col=1,
)

# Row 2 – PSAR as a separate line (for scale/clarity)
fig_psar.add_trace(
    go.Scatter(
        x=view_psar["Date"],
        y=view_psar["PSAR"],
        mode="lines",
        name="PSAR (line)"
    ),
    row=2, col=1
)

fig_psar.update_layout(
    title=f"AAPL – Parabolic SAR Strategy (Zoomed: {start_date.date()} to {end_date.date()})",
    yaxis_title="Price",
    xaxis2_title="Date",
    yaxis2_title="PSAR",
    legend_title="Legend",
    xaxis_rangeslider_visible=False,
    height=700,
)

fig_psar.show()


In [19]:
merged_psar = pd.merge(
    psar_results[["Date", "strategy_equity", "buyhold_equity"]],
    fund[["Date", "fund_equity"]],
    on="Date",
    how="inner"
)

fig_eq_psar = go.Figure()

fig_eq_psar.add_trace(go.Scatter(
    x=merged_psar["Date"],
    y=merged_psar["strategy_equity"],
    mode="lines",
    name="Parabolic SAR Strategy"
))

fig_eq_psar.add_trace(go.Scatter(
    x=merged_psar["Date"],
    y=merged_psar["buyhold_equity"],
    mode="lines",
    name="Buy & Hold AAPL"
))

fig_eq_psar.add_trace(go.Scatter(
    x=merged_psar["Date"],
    y=merged_psar["fund_equity"],
    mode="lines",
    name="TDB908 Fund"
))

fig_eq_psar.update_layout(
    title="Equity Curves – Parabolic SAR Strategy vs AAPL vs TD Nasdaq Index Fund",
    xaxis_title="Date",
    yaxis_title="Portfolio value (start = 1.0)",
    legend_title="Legend",
    xaxis_rangeslider_visible=False,
    height=600,
)

fig_eq_psar.show()


### Summarize all Returns

In [20]:
# Compile all strategy metrics into a summary table

rows = []

# 1) Supertrend strategy
rows.append(
    summarize_strategy(
        "Supertrend",
        super_results["strategy_ret"],
        super_results["strategy_equity"],
    )
)

# 2) Parabolic SAR strategy
rows.append(
    summarize_strategy(
        "Parabolic SAR",
        psar_results["strategy_ret"],
        psar_results["strategy_equity"],
    )
)

# 3) Donchian breakout strategy
rows.append(
    summarize_strategy(
        "Donchian Breakout",
        donch_results["strategy_ret"],
        donch_results["strategy_equity"],
    )
)

# 4) Buy & Hold AAPL (baseline)
#    Use daily_ret + buyhold_equity from any of the results (they’re the same AAPL series)
rows.append(
    summarize_strategy(
        "Buy & Hold AAPL",
        donch_results["daily_ret"],
        donch_results["buyhold_equity"],
    )
)

# 5) TD Nasdaq Index Fund (TDB908)
rows.append(
    summarize_strategy(
        "TDB908 Fund",
        fund["fund_ret"],
        fund["fund_equity"],
    )
)

metrics_df = pd.DataFrame(rows)



In [21]:
metrics_df

,Strategy,Success Rate (%),Total Return (%),Annual Return (%),Sharpe Ratio
0,Supertrend,33.929992,694.111147,23.094541,1.103858
1,Parabolic SAR,32.020684,328.215418,15.702350,0.761736
2,Donchian Breakout,33.174224,598.457696,21.520388,1.035945
3,Buy & Hold AAPL,53.500398,940.566200,26.476644,0.885394
4,TDB908 Fund,54.281163,393.915399,17.392841,0.740231


In [22]:
import plotly.express as px

def plot_metric_bar(metrics_df, metric_col, title, is_percent=True):
    # Sort strategies by the metric (descending)
    df_plot = metrics_df.sort_values(metric_col, ascending=False).copy()
    
    # Horizontal bar chart
    fig = px.bar(
        df_plot,
        x=metric_col,
        y="Strategy",
        orientation="h",
        text=df_plot[metric_col].round(2),
        title=title,
    )
    
    # Put labels at the end of bars
    fig.update_traces(textposition="outside")
    
    # Make sure there is a bit of space to show labels
    max_val = df_plot[metric_col].max()
    fig.update_xaxes(range=[0, max_val * 1.15])
    
    # Axis labels
    fig.update_layout(
        xaxis_title=metric_col,
        yaxis_title="Strategy",
        height=400,
        showlegend=False,
    )
    
    # Optionally format percentages nicely (they're still numeric, just the label)
    if is_percent:
        fig.update_xaxes(ticksuffix="%")
    
    fig.show()

In [23]:
# 1) Annual return comparison
plot_metric_bar(
    metrics_df,
    metric_col="Annual Return (%)",
    title="Annualized Return Comparison – Strategies vs Buy & Hold vs TDB908 Fund",
    is_percent=True
)


In [24]:
# 2) Success rate comparison
plot_metric_bar(
    metrics_df,
    metric_col="Success Rate (%)",
    title="Success Rate (Winning Days) – Strategies vs Benchmarks",
    is_percent=True
)


In [25]:
# 3) Sharpe ratio comparison
plot_metric_bar(
    metrics_df,
    metric_col="Sharpe Ratio",
    title="Sharpe Ratio Comparison – Risk-Adjusted Performance",
    is_percent=False
)

In [26]:
# 4) Total return comparison
plot_metric_bar(
    metrics_df,
    metric_col="Total Return (%)",
    title="Total Return Comparison – Strategies vs Buy & Hold vs TDB908 Fund",
    is_percent=True
)

In [27]:
import pandas as pd
import plotly.graph_objects as go

# Ensure Date columns are datetime & sorted
for df in [super_results, psar_results, donch_results, fund]:
    df["Date"] = pd.to_datetime(df["Date"])
    df.sort_values("Date", inplace=True)

# Build a combined dataframe on common dates
combined = donch_results[["Date", "strategy_equity", "buyhold_equity"]].rename(
    columns={
        "strategy_equity": "Donchian Equity",
        "buyhold_equity": "AAPL Buy & Hold"
    }
)

combined = combined.merge(
    super_results[["Date", "strategy_equity"]].rename(columns={"strategy_equity": "Supertrend Equity"}),
    on="Date",
    how="inner"
)

combined = combined.merge(
    psar_results[["Date", "strategy_equity"]].rename(columns={"strategy_equity": "PSAR Equity"}),
    on="Date",
    how="inner"
)

combined = combined.merge(
    fund[["Date", "fund_equity"]].rename(columns={"fund_equity": "TDB908 Fund Equity"}),
    on="Date",
    how="inner"
)

# Create the equity curve plot
fig_eq_all = go.Figure()

fig_eq_all.add_trace(go.Scatter(
    x=combined["Date"], y=combined["Supertrend Equity"],
    mode="lines", name="Supertrend Strategy"
))

fig_eq_all.add_trace(go.Scatter(
    x=combined["Date"], y=combined["PSAR Equity"],
    mode="lines", name="Parabolic SAR Strategy"
))

fig_eq_all.add_trace(go.Scatter(
    x=combined["Date"], y=combined["Donchian Equity"],
    mode="lines", name="Donchian Breakout Strategy"
))

fig_eq_all.add_trace(go.Scatter(
    x=combined["Date"], y=combined["AAPL Buy & Hold"],
    mode="lines", name="Buy & Hold AAPL"
))

fig_eq_all.add_trace(go.Scatter(
    x=combined["Date"], y=combined["TDB908 Fund Equity"],
    mode="lines", name="TDB908 Fund"
))

fig_eq_all.update_layout(
    title="Equity Curves – All Strategies vs AAPL Buy & Hold vs TDB908 Fund",
    xaxis_title="Date",
    yaxis_title="Portfolio Value (start = 1.0)",
    legend_title="Strategy",
    xaxis_rangeslider_visible=False,
    height=600,
)

fig_eq_all.show()
